In [16]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_circular_economy as mc
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)

<module 'support_functions' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py'>

In [21]:
sa.model_attributes.dict_attributes["cat_wastewater_treatment"].table

,category_name,cat_wastewater_treatment,description,data_source,hyperlink,notes,default_tow_removal_fraction,default_tow_removal_fraction_lower_bound,default_tow_removal_fraction_upper_bound
0,Untreated No Sewarage,untreated_no_sewerage,Wastewater that is untreated without sewerage....,NaN,NaN,NaN,0.000,0.00,0.10
1,Untreated With Sewerage,untreated_with_sewerage,"Wastewater that is sent to sewers, but is not ...",NaN,NaN,NaN,0.000,0.00,0.10
2,Centralized Aerobic Treatment,treated_cetralized_aerobic,| Wastewater that is treated in centralized a...,NaN,NaN,NaN,0.720,0.25,0.95
3,Anaerobic Treatment,treated_anaerobic,| Wastewater that is treated in anaerobic pla...,NaN,NaN,NaN,0.720,0.25,0.95
4,Septic,treated_septic,"Septic tanks, which are a common treatment opt...",NaN,NaN,NaN,0.625,0.50,0.60
5,Latrine,treated_latrine,Latrines,NaN,NaN,NaN,0.430,0.05,1.00


In [25]:
class CircularEconomy:

    def __init__(self, attributes: ds.ModelAttributes):

        self.model_attributes = attributes
        self.required_dimensions = self.get_required_dimensions()
        self.required_subsectors, self.required_base_subsectors = self.get_required_subsectors()
        self.required_variables, self.output_variables = self.get_ce_input_output_fields()

        ##  set some model fields to connect to the attribute tables

        # domestic solid waste model variables
        self.modvar_waso_boc = "BOC"
        self.modvar_waso_frac_dom_compo = "Composition Fraction Domestic Solid Waste"
        self.modvar_waso_frac_ind_compo = "Composition Fraction Industrial Solid Waste"
        self.modvar_waso_dem_energy_recycle = "Energy Demand to Recycle Waste"
        self.modvar_waso_dsw_pc_init = "Initial Per Capita Annual Domestic Solid Waste Generated"
        self.modvar_waso_dsw_elast = "Elasticity of Waste Produced to GDP/Capita"
        self.modvar_waso_dsw_elast = "Initial Per VA Solid Industrial Waste Generated"
        self.modvar_waso_edrf = "Virgin Production Energy Demand Reduction Factor"
        self.modvar_waso_frac_dsw_nr_burned = "Fraction of Non-Recycled Domestic Solid Waste Burned"
        self.modvar_waso_frac_dsw_nr_landfilled = "Fraction of Non-Recycled Domestic Solid Waste Landfilled"
        self.modvar_waso_frac_dsw_nr_open = "Fraction of Non-Recycled Domestic Solid Waste Open Dumps"
        self.modvar_waso_frac_dsw_recycle = "Fraction of Waste Recycled"   
        self.modvar_waso_frac_landfill_gas_captured = "Fraction of Landfill Gas Captured at Landfills"
        self.modvar_waso_k = "K"
        
        
        # economy and general variables
        self.modvar_econ_gdp = "GDP"
        self.modvar_econ_va = "Value Added"
        self.modvar_gnrl_area = "Area of Country"
        self.modvar_gnrl_occ = "National Occupation Rate"
        self.modvar_gnrl_subpop = "Population"
        self.modvar_gnrl_pop_total = "Total Population"

        ##  MISCELLANEOUS VARIABLES

        self.time_periods, self.n_time_periods = self.model_attributes.get_time_periods()

        # TEMP:SET TO DERIVE FROM ATTRIBUTE TABLES---
        self.landfill_gas_frac_methane = 0.5


    ##  FUNCTIONS FOR MODEL ATTRIBUTE DIMENSIONS

    def check_df_fields(self, df_ce_trajectories):
        check_fields = self.required_variables
        # check for required variables
        if not set(check_fields).issubset(df_ce_trajectories.columns):
            set_missing = list(set(check_fields) - set(df_ce_trajectories.columns))
            set_missing = sf.format_print_list(set_missing)
            raise KeyError(f"Circular Economy projection cannot proceed: The fields {set_missing} are missing.")


    def get_required_subsectors(self):
        subsectors = list(sf.subset_df(self.model_attributes.dict_attributes["abbreviation_subsector"].table, {"sector": ["Circular Economy"]})["subsector"])
        subsectors_base = subsectors.copy()
        subsectors += ["Economy", "General"]
        return subsectors, subsectors_base

    def get_required_dimensions(self):
        ## TEMPORARY - derive from attributes later
        required_doa = [self.model_attributes.dim_time_period]
        return required_doa

    def get_ce_input_output_fields(self):
        required_doa = [self.model_attributes.dim_time_period]
        required_vars, output_vars = self.model_attributes.get_input_output_fields(self.required_subsectors)
        return required_vars + self.get_required_dimensions(), output_vars

    

    

In [26]:
circ_econ = CircularEconomy(sa.model_attributes)






In [30]:
sa.model_attributes.build_varlist("Wastewater Treatment", variable_type = "input")

['ef_trww_tonne_ch4_per_tonne_bod',
 'ef_trww_tonne_ch4_per_tonne_bod',
 'ef_trww_tonne_ch4_per_tonne_bod',
 'ef_trww_tonne_ch4_per_tonne_bod',
 'ef_trww_tonne_ch4_per_tonne_bod',
 'ef_trww_tonne_ch4_per_tonne_bod',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'mcf_trww_$CAT-WASTEWATER-TREATMENT',
 'frac_trww_tow_removed_treated_anaerobic',
 'frac_trww_tow_removed_treated_cetralized_aerobic',
 'frac_trww_tow_removed_treated_latrine',
 'frac_trww_tow_removed_treated_septic',
 'frac_trww_tow_removed_untreated_no_sewerage',
 'frac_trww_tow_removed_untreated_with_sewerage']

In [ ]:
#########################
###                   ###
###    SOLID WASTE    ###
###                   ###
#########################


##   ESTIMATE WASTE GENERATED BY CATEGORY


##################################
#    RECYCLING AND COMPOSTING    #
##################################


##   ESTIMATE TOTAL WASTE RECYLCED + COMPOSTED/BIOGAS


##   FOR WASTE THAT IS RECYCLED, GET TOTALS TO SEND TO IPPU FOR PROCESS EMISSIONS


##   FOR COMPOST/BIOGAS, ESTIMATE EMISSIONS AND GET TOTAL BIOGAS AVAILABLE AS A FUEL


###################
#    LANDFILLS    #
###################

##   ESTIMATE TOTAL WASTE THAT IS LANDFILLED


##   ESTIMATE EMISSIONS USING FIRST-ORDER DECAY MODEL


##   ESTIMATE LANDFILL GAS (METHANE) THAT IS CAPTURED


##   ESTIMATE MASS OF LANDFILL GAS THAT IS AVAILABLE AS FUEL 



#######################################
#    INCINERATION AND OPEN BURNING    #
#######################################

##  TOTAL WASTE OPEN BURNED


##  EMISSIONS FROM OPEN BURNING


##  FRACTION OF 




##########################
#    UNCOLLECTED WASTE   #
##########################

##  TOTAL WASTE UNCOLLECTED


##  



########################################################################
#    INCREASED TRANSPORTATION DEMAND FOR HEAVY TRUCKS FOR COLLECTION   #
########################################################################

# GET VMT PER CAPITA




In [29]:
importlib.reload(sa)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python


<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/setup_analysis.py'>

In [4]:
##########################
###                    ###
###    LIQUID WASTE    ###
###                    ###
##########################

##  GET INITIAL WW GENERATED + BASED ON BOD/PERSON, SET IMPLIED FRACTION OF BOD/M3 WW


##  CALCULATE TREATMENT BY TYPE



##  UNTREATED, NO SEWAGE: 



##  UNTREATED, SEWERAGE:



##  CENTRALIZED AEROBIC:



##  ANAEROBIC TREATMENT:



##  SEPTIC AND LATRINE




0.9527145359019265